Importando as bibliotecas

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

Carregando a base de dados a partir do arquivo

In [3]:
database = pd.read_excel('/content/ArquivoExemplo.xlsx')

Exibindo as primeiras linhas da base de dados

In [4]:
print(database)

   História do crédito Dívida Garantias        Renda Anual     Risco
0                 Ruim   Alta   Nenhuma             <15000      Alto
1         Desconhecida   Alta   Nenhuma  >=15000 a <=35000      Alto
2         Desconhecida  Baixa   Nenhuma  >=15000 a <=35000  Moderado
3         Desconhecida  Baixa   Nenhuma             >35000      Alto
4         Desconhecida  Baixa   Nenhuma             >35000     Baixo
5         Desconhecida  Baixa  Adequada             >35000     Baixo
6                 Ruim  Baixa   Nenhuma             <15000      Alto
7                 Ruim  Baixa  Adequada             >35000  Moderado
8                  Boa  Baixa   Nenhuma             >35000     Baixo
9                  Boa   Alta  Adequada             >35000     Baixo
10                 Boa   Alta   Nenhuma             <15000      Alto
11                 Boa   Alta   Nenhuma  >=15000 a <=35000  Moderado
12                 Boa   Alta   Nenhuma             >35000     Baixo
13                Ruim   Alta   Ne

Convertendo as variáveis categóricas em numéricas 

In [5]:
le = LabelEncoder()
database['História do crédito'] = le.fit_transform(database['História do crédito'])
database['Dívida'] = le.fit_transform(database['Dívida'])
database['Garantias'] = le.fit_transform(database['Garantias'])
database['Renda Anual'] = le.fit_transform(database['Renda Anual'])
database['Risco'] = le.fit_transform(database['Risco'])

Separando as variáveis independentes(X) das dependentes(Y)


In [6]:
X = database.drop('Risco', axis=1)
Y = database['Risco']

Definir o grid de parâmetros a serem testados:

In [7]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

Criar um objeto GridSearchCV com o modelo RandomForestClassifier e o grid de parâmetros a serem testados:

In [8]:
rfc = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

Executar o GridSearchCV com os dados de treinamento:

In [9]:
grid_search.fit(X, Y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [50, 100, 200]})

Imprimir o melhor conjunto de parâmetros encontrados:

In [10]:
print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 50}


Criar um novo modelo RandomForestClassifier com os melhores parâmetros encontrados e treinar com a base de dados completa:

In [11]:
best_rfc = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],
                                   max_features=grid_search.best_params_['max_features'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   criterion=grid_search.best_params_['criterion'])
best_rfc.fit(X, Y)

RandomForestClassifier(max_depth=4, n_estimators=50)

Fazer previsões com o modelo treinado:

In [14]:
novos_dados = pd.DataFrame({
    'História do crédito': ['Ruim', 'Boa'],
    'Dívida': ['Baixa', 'Alta'],
    'Garantias': ['Nenhuma', 'Adequada'],
    'Renda Anual': ['<15000', '>35000']
})
novos_dados = novos_dados.apply(le.fit_transform)
previsoes = best_rfc.predict(novos_dados)
print(previsoes)

[0 1]
